In [11]:
import json
import copy
import math
import numpy as np
#from Scripts.Stock_Filter import load_merged_data
from statistics import mean , stdev
from scipy.stats import norm
#-------------------------------
class Node:
    """
    Represents a node in a graph, typically used in search algorithms like A* or BFS.

    Attributes:
        value (float): The value associated with this node (e.g., stock price).
        heuristic (float): The heuristic value estimating the cost to reach the goal.
        symbol (str): A unique identifier for the node (e.g., stock symbol).
        action (str): The action associated with this node (e.g., 'buy', 'sell', 'hold').
        neighbors (list): A list of neighboring nodes connected to this node.
        parent (Node): The parent node in the search tree.
    """

    def __init__(self, stock, heuristic=0, parent=None, neighbors=None):
        """
        Initializes a Node object.

        :param stock: A dictionary or object containing stock details (symbol, action, value).
        :param heuristic: The heuristic value for the node.
        :param parent: The parent node in the search tree (default is None).
        :param neighbors: A list of neighboring nodes (default is an empty list).
        """
        self.value = stock.get("value", 0)  # Cost of the stock (default to 0 if not provided)
        self.heuristic = heuristic  # Heuristic value
        self.symbol = stock.get("symbol", "unknown")  # Stock symbol (default to 'unknown' if not provided)
        self.action = stock.get("action", "none")  # Action (default to 'none' if not provided)
        self.neighbors = neighbors if neighbors is not None else []  # Neighbors list
        self.parent = parent  # Parent node

    @property
    def set_cop_stock(self):
        """Returns a dictionary representation of the stock data."""
        return {
            "symbol": self.symbol,
            "action": self.action,
            "value": self.value
        }

    def add_neighbor(self, neighbor):
        """
        Adds a neighboring node to this node's neighbors list.

        :param neighbor: The neighboring node to add.
        """
        if neighbor not in self.neighbors:
            self.neighbors.append(neighbor)

    def remove_neighbor(self, neighbor):
        """
        Removes a neighboring node from this node's neighbors list.

        :param neighbor: The neighboring node to remove.
        """
        if neighbor in self.neighbors:
            self.neighbors.remove(neighbor)

    def set_parent(self, parent):
        """Sets the parent node."""
        self.parent = parent

    def get_Key(self):
        """Returns a unique key for the node combining symbol and action."""
        return f"{self.symbol}_{self.action}"

    def get_heu(self):
        """Returns the heuristic value of the node."""
        return self.heuristic

    def get_copy(self):
        """Returns a copy of the node."""
        copy_node = Node(self.set_cop_stock, self.heuristic)
        return copy_node

    def __repr__(self):
        """
        String representation of the node for debugging purposes.

        :return: A string summarizing the node's attributes.
        """
        return (
            f"Node(symbol={self.symbol}, action={self.action}, value={self.value:.2f}, "
            f"heuristic={self.heuristic:.2f}, neighbors={len(self.neighbors)})"
        )
class StockItem:
    """
    Represents an individual stock item with a stock name and the amount owned.
    """
    def __init__(self, stock, amt=1):
        """
        Initializes a stock item.

        :param stock: The name of the stock.
        :param amt: The amount of the stock owned (default is 1).
        """
        self.stock = stock
        self.amt = amt

class Stocks:
    """
    Represents a stock portfolio where users can buy and sell stocks using available funds.
    """
    def __init__(self, funds, stocks=None):
        """
        Initializes the Stocks portfolio.

        :param funds: Initial amount of funds available.
        :param stocks: Optional list of StockItem objects representing owned stocks.
        """
        self.funds = funds
        self.stocks = stocks if stocks is not None else []
        self.value  = 0

    def buy(self, stock, price):
        """
        Purchases a stock if sufficient funds are available.

        :param stock: The name of the stock to buy.
        :param price: The price of the stock.
        """
        if self.funds < price:
            print(f"Insufficient funds to buy {stock}. Available: ${self.funds:.2f}, Required: ${price:.2f}")
            return

        self.funds -= price  # Deduct the stock price from funds
        found = False

        # Check if the stock is already owned
        for el in self.stocks:
            if el.stock == stock:
                el.amt += 1  # Increase the stock amount
                found = True
                break

        # If stock is not found, add a new entry
        if not found:
            self.stocks.append(StockItem(stock))

        print(f"Bought 1 share of {stock} for ${price:.2f}. Remaining funds: ${self.funds:.2f}")

    def sell(self, stock, price):
        """
        Sells a stock if the user owns it.

        :param stock: The name of the stock to sell.
        :param price: The price of the stock at the time of selling.
        """
        for el in self.stocks:
            if el.stock == stock:
                el.amt -= 1  # Reduce stock amount
                self.funds += price  # Add the selling price to funds

                # Remove the stock from the list if the amount reaches zero
                if el.amt == 0:
                    self.stocks.remove(el)

                print(f"Sold 1 share of {stock} for ${price:.2f}. Updated funds: ${self.funds:.2f}")
                return

        print(f"Cannot sell {stock}: Not owned.")

    def has_stock(self, stock):
        """
        Checks if a stock exists in the portfolio.

        :param stock: The name of the stock to check.
        :return: True if the stock is in the portfolio, False otherwise.
        """
        return any(el.stock == stock for el in self.stocks)
    def get_stock (self , stock):
      for st in self.stocks :
        if st.stock == stock :
          return st.amt
      return 0
    def eval_por(self, stocks):
      prix = 0
      for stock in stocks :
        if self.has_stock(stock["symbol"]):
          prix = prix + self.get_stock(stock)
      return prix

    def percentage(self, stock):
        """
        Calculates the percentage of a specific stock in the portfolio.

        :param stock: The name of the stock to calculate percentage for.
        :return: The percentage of the stock in the portfolio, or 0 if not found or total is zero.
        """
        if not self.has_stock(stock):
            return 0.0

        total = sum(el.amt for el in self.stocks)  # Total number of shares
        if total == 0:  # Prevent division by zero
            return 0.0

        for el in self.stocks:
            if el.stock == stock:
                return el.amt / total
        return 0.0  # Fallback, though has_stock ensures this shouldn't happen

    def display_portfolio(self):
        """
        Displays the current stock holdings and available funds.
        """
        print(f"\nAvailable funds: ${self.funds:.2f}")
        if not self.stocks:
            print("Stock Portfolio is empty.")
        else:
            print("Stock Portfolio:")
            for stock in self.stocks:
                print(f"- {stock.stock}: {stock.amt} shares")
        print("-" * 30)
#---------------------------------


def CalculateScore(statistical_variable, value):
    """
    Normalizes the score of a statistical variable (standard deviation, VaR, CVaR)
    to a value between 0 and 1 based on its risk level.
    """
    score_values = {"low": 0.3, "high": 0.7}
    level = LevelOfStatisticalVariable(statistical_variable, value)

    if "low" in level:
        min_val, max_val = level["low"]
        denominator = max_val - min_val if max_val != min_val else 1
        score = score_values["low"] * ((value - min_val) / denominator)

    elif "moderate" in level:
        min_val, max_val = level["moderate"]
        denominator = max_val - min_val if max_val != min_val else 1
        score = score_values["low"] + (score_values["high"] - score_values["low"]) * ((value - min_val) / denominator)

    elif "high" in level:
        min_val, max_val = level["high"]
        denominator = max_val - min_val if max_val != min_val else 1
        score = score_values["high"] + (1 - score_values["high"]) * ((value - min_val) / denominator)

    else:
        raise ValueError("Unrecognized level from LevelOfStatisticalVariable")

    return round(score, 4)


def CalculateTotalRisk(volatility_val,Var_val,Cvar_val):
    """
    calculates the total risk of the portfolio .
    """
    volatility_score = CalculateScore("standard_deviation",volatility_val)
    Var_score = CalculateScore("Var",Var_val)
    Cvar_score = CalculateScore("Cvar",Cvar_val)
    total_risk = (0.5 * volatility_score) + (0.4 * Var_score) + (0.1 * Cvar_score)
    return total_risk


def LevelOfStatisticalVariable(statistical_variable,value):
    """
      determines which level we can consider the statistical variable according to its value .
    """
    standard_deviation = [{"low" : (0,0.05)},
                          {"moderate" : (0.05,0.15)},
                          {"high" : (0.15,1)}]
    Var = [{"low" : (0,0.02)},
          {"moderate" : (0.02,0.05)},
          {"high" : (0.05,1)}]
    standard_deviation = [{"low" : (0,0.03)},
                          {"moderate" : (0.03,0.06)},
                          {"high" : (0.06,1)}]
    if statistical_variable == "standard_deviation":
      if value <= 0.05:
        return standard_deviation[0]
      if value > 0.05 and value < 0.15 :
        return standard_deviation[1]
      else:
        return standard_deviation[2]

    if statistical_variable == "Var":
      value = value * (-1)
      if value <= 0.02:
        return standard_deviation[0]
      if value > 0.02 and value < 0.05 :
        return standard_deviation[1]
      else:
        return standard_deviation[2]

    if statistical_variable == "Cvar":
      value = value * (-1)
      if value <= 0.03:
        return standard_deviation[0]
      if value > 0.03 and value < 0.06 :
        return standard_deviation[1]
      else:
        return standard_deviation[2]


def calculate_returns(prices):
    """Calculate daily returns from a list of prices."""
    returns = np.diff(prices) / prices[:-1]
    return returns


def get_var_and_cvar(stock, confidence_level=0.95):
    """
    Calculate Value at Risk (VaR) and Conditional Value at Risk (CVaR).

    Args:
        stock (dict): Dictionary containing 'symbol' and 'prices'.
        confidence_level (float): Confidence level for VaR (e.g., 0.95 for 95%).

    Returns:
        tuple: VaR and CVaR values.
    """
    prices =  stock["prices"]
    returns = calculate_returns(prices)

    z_score = norm.ppf(1 - confidence_level)  # Z-score for the given confidence level
    mean = np.mean(returns)
    std_dev = np.std(returns)
    var = mean - z_score * std_dev  # VaR formula

    # Calculate CVaR as the mean of losses exceeding VaR
    losses_beyond_var = returns[returns <= var]
    cvar = np.mean(losses_beyond_var) if len(losses_beyond_var) > 0 else var

    return var, cvar  # Return raw VaR/CVaR (negative for losses)



def GetStandarddeviation(node,user_constraints):  # we must modify the parameter to be self
    """
      calculates the standard deviation of the stock that represents the volatility of it .
      user constraint is a dictionary where {"preferred_risk":"-","investment_duration":"-"}
    """
    if user_constraints["investment_duration"] == "short_term":
      with open('Data\StockFilterData\stocks_consecutive_day_volatility.json','r') as file :
        data = json.load(file)
    else:
      with open('Data\StockFilterData\stocks_consecutive_day_annual_volatility.json','r') as file :
        data = json.load(file)
    symbol = node.stock
    stock_dict = next((item for item in data if item['symbol'] == symbol),None)  #next() finds the first match
    volatilities = stock_dict['volatilities']
    std_value = np.std(volatilities,ddof=1) #ddof is the data degree of freedom
    return std_value


def GetPortfolioStatVariables(node, portfolio): #update other functions to take a stock not a node
    """
	   calculates the statistical variables (standard deviation , var , cvar) for the portfolio .
    """
    quadratic_sum = 0
    var_sum = 0
    cvar_sum =0
    stat_variables = dict()
    stocks = portfolio.stocks
    for stock in stocks :
      std_dev = GetStandarddeviation(stock)
      var = get_var_and_cvar(stock)[0]
      cvar = get_var_and_cvar(stock)[1]
      weight = GetWeight(node,stock,portfolio)
      quadratic_sum += (weight ** 2) * (std_dev ** 2)
      var_sum += weight *  var
      cvar_sum += weight * cvar
    stat_variables["standard_deviation"] = math.sqrt(quadratic_sum)
    stat_variables["Var"] = var_sum
    stat_variables["Cvar"] = cvar_sum
    return stat_variables


def GetWeight(node,stock,portfolio):
    """
    computes the allocation amount of stock in the portfolio (budget based)
    """
    weight = (node.value*(stock.amt)) / portfolio.funds
    return weight


def allocation (node, portfolio):#-----------------------------
    """
    computes the allocation amount of stock in the portfolio (total return based)
    """
    value = node.stock["price"]* node.value
    port_return = node.eval_por(portfolio.stocks)
    return value/port_return



def stock_class_limit_not_violated(portfolio , portfolio_type, mainstock,allocation):
    """
    checks if a class of stocks percentage is violating the pereferred stock type requirment (retrun False) or not (return True)
    """
    risky ,moderate ,lowrisk = classify_stock_risk(portfolio)
    risk_allocation_limit = {
     "aggressive" : {
         "high":   (0.50 , 0.70),
         "moderate":   (0.20 , 0.40),
         "low":   (0.00 , 0.15),
      },
      "moderate" : {
         "high":   (0.20 , 0.35),
         "moderate":   (0.4 , 0.50),
         "low":   (0.15 , 0.30),
      },
      "conservative" : {
         "high":   (0.00 , 0.10),
         "moderate":   (0.20 , 0.40),
         "low":   (0.50 , 0.70),
      }
      }
    if portfolio_type == "aggressive":
       if mainstock in risky :#imported from data
           return range_limit(risk_allocation_limit["aggressive"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["aggressive"]["moderate"],allocation)
       elif mainstock in lowrisk :#imported from data
           return range_limit(risk_allocation_limit["aggressive"]["low"],allocation)
    elif portfolio_type == "moderate":
       if mainstock in risky :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["moderate"],allocation)
       elif mainstock in lowrisk :#imported from data
           return range_limit(risk_allocation_limit["moderate"]["low"],allocation)
    elif portfolio_type == "conservative":
       if mainstock in risky :#imported from data
           return range_limit(risk_allocation_limit["conservative"]["high"],allocation)
       elif mainstock in moderate :#imported from data
           return range_limit(risk_allocation_limit["conservative"]["moderate"],allocation)
       elif mainstock in lowrisk :#imported from data
           return range_limit(risk_allocation_limit["conservative"]["low"],allocation)


def diversification_limit_not_violated( portfolio_type="aggressive", allocation=0.1):
   """
   checks if the diversification limit is violated (retrurns false) or not (returns true)
   """
   diversification_limit = {
         "aggressive":   (0.00 , 0.25),
         "moderate":   (0.00 , 0.15),
         "conservative":   (0.00 , 0.10),
         }
   if portfolio_type=="aggressive":
       return range_limit(diversification_limit["aggressive"] ,allocation)
   elif portfolio_type=="moderate":
       return range_limit(diversification_limit["moderate"] ,allocation)
   elif portfolio_type=="conservative":
       return range_limit(diversification_limit["conservative"] ,allocation)



def range_limit (tuple, value):
  """
  checks if a value is in the interval [tuple[0],tuple[1]]
  """
  x,y = tuple
  if x <= value <= y:
     return True
  else:
     return False



def TotalRiskLevel(risk_score):
  """
  determines the level of portfolio from the risk_score ("conservative" ,"moderate" or "aggressive" )
  """
  if risk_score<= 0.4 :
    return "conservative"
  elif risk_score <= 0.7:
    return "moderate"
  else:
    return "aggressive"

def number_of_invested(portfolio):
   """
   number of stocks that are being invested in in the portfolio
   """
   count = 0
   for stock in portfolio.stocks :
      if stock.amt >0 :
         count += 1
   return count

def calculate_cv(portfolio):
    """
    calculate the coeficient of variation of the number of shares
    """
    shares = [stock.amt for stock in portfolio.stocks]
    std = stdev(shares)
    meanshares = mean(shares)
    return std / meanshares

def expand(node, mainstock, portfolio , user_constraints):  # replace the node with self
    """
     user constraint is a dictionary where {"preferred_risk":"-","investment_duration":"-"}
    """
    expand = []
    default  = {"id": node.symbol , "hold": 0}
    expand.append(default)
    buy_portfolio=None
    sell_portfolio=None
    with open('Data\StockFilterData\multiple_stocks_ratios.json', 'r') as file:
      data = json.load(file)
    for stockinfo in data :
       if stockinfo["symbol"] == mainstock.stock:
           for stock in portfolio.stocks :
               if stock.stock == mainstock.stock:
                  stock.amt+=1
                  buy_portfolio=copy.deepcopy(portfolio)
                  stock.amt-=2
                  if stock.amt >= 0 :
                      sell_portfolio = copy.deepcopy(portfolio)
                  stock.amt+=1 #back to its orignal value
    stat_variables = GetPortfolioStatVariables(node , buy_portfolio)
    std_value = stat_variables["standard_deviation"]
    var = stat_variables["Var"]
    cvar = stat_variables["Cvar"]


    #-----------------------------------------
    #Computing weight based on the variety of stocks in the portfolio
    #-----------------------------------------
    cv = calculate_cv(portfolio)
    numinv = number_of_invested(portfolio)
    if cv <= 0.5 and number_of_invested >=6 :
       w= allocation(node, portfolio)
    else :
       w= GetWeight(node, mainstock,portfolio)
    #-----------------------------------------
    #Buy action check
    #-----------------------------------------
    if portfolio.funds >= node.value  :

      #diversification and stock class check
      portfolio_type = user_constraints["preferred_risk"]
      investement_duration = user_constraints["investment_duration"]
      portfolio_risk_score = CalculateTotalRisk(std_value,var,cvar)
      portfolio_risk = TotalRiskLevel(portfolio_risk_score)
      if  portfolio_risk == user_constraints["preferred_risk"]  and diversification_limit_not_violated(portfolio_type,w) and stock_class_limit_not_violated( portfolio , portfolio_type, mainstock,w) and investment_term_constraint( investement_duration,portfolio):
        expand.append({"id": node.symbol,"buy":-node.value})

    #-----------------------------------------
    #Sell action check
    #-----------------------------------------
    if sell_portfolio :
       portfolio_risk_score = CalculateTotalRisk(std_value,var,cvar)
       portfolio_risk = TotalRiskLevel(portfolio_risk_score)
       if portfolio_risk == user_constraints["preferred_risk"]  and diversification_limit_not_violated(portfolio_type,w) and stock_class_limit_not_violated( portfolio , portfolio_type, mainstock,w) and investment_term_constraint( investement_duration,portfolio):
           expand.append({"id": node.symbol,"sell":+node.value})
    return expand


def classify_stock_risk(portfolio):
    """
    classifies the portfolio stocks to three lists of risk levels
    """
    Risky=[]
    Moderate=[]
    LowRisk=[]

    with open ('Data\StockFilterData\multiple_stocks_ratios.json', 'r') as f :
       data = json.load(f)
    vol_list= extract_ratio_list (data,"volatility")
    cu_list= extract_ratio_list (data,"currentRatioTTM")
    dept_list= extract_ratio_list (data,"dept_to_equity")


    for stockitem in portfolio.stocks :
        stock_ratios = next((item for item in data if item["symbol"]==stockitem.stock),None)
        volatility = stock_ratios.get("volatility", 1.0)
        cu_ratio = stock_ratios.get("currentRatioTTM",15.0)
        debt_equity = stock_ratios.get("dept_to_equity",1.0)

        vol_nor = normalize(volatility,min(vol_list),max(vol_list))
        cu_nor = normalize(cu_ratio,min(cu_list),max(cu_list))
        dept_nor = normalize(debt_equity,min(dept_list),max(dept_list))

        class_val = vol_nor*0.7-cu_nor*0.3

        if class_val<0.3:
          LowRisk.append(stockitem)
        elif class_val<0.6:
          Moderate.append(stockitem)
        else:
          Risky.append(stockitem)
    return Risky,Moderate,LowRisk


def extract_ratio_list(data, key ,default=0.0 ):
    """
    extracts a list of a specefic ratio from data
    """
    return [item.get(key, default) for item in data if item.get(key) is not None   ]


def normalize (value, min , max):
    """
    normalize a value to the range [0,1]
    """
    if min == max:
       return 0.0
    return (value -min) / (max - min)

def investment_term_constraint( investement_duration,portfolio):
    df = load_merged_data()
    short_term = get_top_short_term(df, 30)
    long_term = get_top_long_term(df, 30)
    shortcount=0
    longcount=0
    for stock in portfolio.stocks :
       if stock.stock in  short_term :
           shortcount+=1
       else :
           longcount+=1

    total_stocks= shortcount+longcount

    if total_stocks == 0:
        return True

    if investement_duration == "short" :
        if longcount/total_stocks > 0.2 :
          return False
    else :
        if shortcount/total_stocks > 0.2 :
          return False
    return True



def get_top_short_term(df, top_n=10):
    """
    returns the best top_n short term stocks from data
    """
    df2 = df.dropna(subset=["expected_return", "volatility", "1M_pct_change"]).copy()
    df2.loc[:, "score"] = (
        df2["expected_return"].rank(ascending=False) +
        df2["1M_pct_change"].rank(ascending=False) +
        df2["volatility"].rank(ascending=True)
    ).round(6)
    df3 = df2.reset_index().sort_values(
        by=["score", "symbol"],
        ascending=[False, True],
        kind="mergesort"
    )
    return df3["symbol"].tolist()[:top_n]


def get_top_long_term(df, top_n=10):
    """
    returns the best top_n long term stocks from data
    """
    df2 = df.dropna(subset=[
        "returnOnEquityTTM",
        "freeCashFlowPerShareTTM",
        "netProfitMarginTTM",
        "pegRatioTTM",
        "debtRatioTTM"
    ]).copy()
    df2.loc[:, "score"] = (
        df2["returnOnEquityTTM"].rank(ascending=False) +
        df2["freeCashFlowPerShareTTM"].rank(ascending=False) +
        df2["netProfitMarginTTM"].rank(ascending=False) +
        df2["pegRatioTTM"].rank(ascending=True) +
        df2["debtRatioTTM"].rank(ascending=True)
    ).round(6)
    df3 = df2.reset_index().sort_values(
        by=["score", "symbol"],
        ascending=[False, True],
        kind="mergesort"
    )
    return df3["symbol"].tolist()[:top_n]
